# Cross-resonance gate between transmon and fluxonium: linear approximation

In this notebook we study the cross-resonance effect between a transmon and a fluxonium in the strong drive regime.

In [1]:
import numpy as np
import scipy.integrate
import time
import qutip as qtp
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pysqkit
from pysqkit import QubitSystem, Qubit
from pysqkit.util.phys import temperature_to_thermalenergy
import pysqkit.util.transformations as trf
from pysqkit.util.linalg import get_mat_elem
from pysqkit.solvers.solvkit import integrate
from matplotlib.colors import LogNorm
from pysqkit.util.hsbasis import weyl_by_index
from pysqkit.solvers import solvkit
import qutip
import itertools
from functools import partial
import multiprocessing
from typing import List, Dict, Callable
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
#import util_cr as util
import copy
import json
import cmath

from IPython.display import display, Latex

The gate is based on the following driven Hamiltonian of a fluxonium and a transmon capacitively coupled

$$H = H^{(0)}_{T, 1} + H^{(0)}_{T, 2} + V + H_{drive},$$

$$H^{(0)}_{T, k} =  \hbar \omega_{T, k} b_k^{\dagger}b_k + \hbar \frac{\delta_{T, k}}{2} b_k^{\dagger} b_k^{\dagger}b_k b_k,$$

$$V = J_C q_{T1} q_{T2},$$

$$\frac{H_{drive}}{h} = \left( \varepsilon_{T1} q_{T1} + \varepsilon_{T2} q_{T2} \right) \cos(2\pi f_d t).$$

The (approximate) charge operator for the transmon in terms of annihilation and creation operators reads

$$
q_{Tk} = i \biggl( \frac{E_{J,k}}{32 |\delta_{T,k} |} \biggr)^{1/4} (b_k^{\dagger} - b_k) = i q_{ZPF} (b_k^{\dagger} - b_k),
$$
where 
$$
E_{J, k} = \hbar \biggl(\frac{\omega_{T, k} - \delta_{T, k}}{8 | \delta_{T, k} |} \biggr)^2.
$$

In the following we introduce relaxation due to dielectric loss and pure dephasing on each qubit as error channels. 

The idea behind the cross-resonance gate is to drive one of the qubit at the frequency of the other using the fact that due to the coupling we have a $Z X$ term in the Hamiltonian. In particular, we will explore the idea of driving the fluxonium at the transmon frequency. The opposite idea to drive the transmon at the fluxonium frequency does not seem that promising. However, one might evaluate using Raman transitions.

# Introducing the qubits and the coupled system

In [2]:
with open('../flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [3]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

p_set = "CR_3"

#Transmon
levels_t = 3
transm_t = pysqkit.qubits.SimpleTransmon(
    label='T1', 
    max_freq=parameters_set[p_set]["max_freq_t"] , 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set]["dephasing_times_t"]
)

#Fluxonium
levels_f = 4

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"], 
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], 
    env_thermal_energy=thermal_energy,
    dephasing_times= parameters_set[p_set]["dephasing_times_f"] #ns/2*np.pi 
)
flx.diagonalize_basis(levels_f)
eig_flx = flx.eig_energies()

# We also add a drive on the fluxonium
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cr_drive_f',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_f - d_comp

jc = parameters_set[p_set]["jc"]
coupled_sys = transm_t.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)

states_label = coupled_sys.all_state_labels()
states_dict = coupled_sys.states_as_dict(as_qobj=True)

In [4]:
state_label = ["00", "01", "10", "11"]
comp_states = {}
for label in state_label:
    state_tmp = coupled_sys.state(label)[1]
    loc = np.argmax(np.abs(state_tmp))
    phase = cmath.phase(state_tmp[loc])
    state_tmp = np.exp(-1j*phase)*state_tmp
    comp_states[label] = state_tmp

The basis principle behind the gate is illustrated in the following figure

The figure shows the energy of the bare levels. The orange arrows indicate the non-zero matrix elements between the (bare) levels provided by the capacitive coupling $V$: the more shaded the arrow the weaker the coupling. 
We look at the following quantities : 

$$\zeta_{ZZ} = E_{00} + E_{11} - E_{01} - E_{10}$$
$$\mu_{YZ} = \lvert \langle 10 | q_{F} | 00 \rangle - \langle 11 | q_{F} |01 \rangle \rvert $$
$$\mu_{Y} = \lvert \langle 10 | q_{F} | 00 \rangle + \langle 11 | q_{F} |01 \rangle \rvert $$

The second term quantifies the gate speed. Notice that if the phase of the drive is $0$ one hase a $YZ$ term from the charge operator. 

For the given parameters we have

In [5]:
def zz(system: QubitSystem) -> float:
    xi_zz = system.state('00')[0] + system.state('11')[0] \
        - system.state('01')[0] - system.state('10')[0]
    return xi_zz

def mu_yz_control(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'])
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 - yz1))/2

def mu_yi_control(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'] )
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 + yz1))/2

In [6]:
display(Latex(r'$\xi_{{ZZ}}/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*zz(coupled_sys))))
q_op = coupled_sys["F"].charge_op()
display(Latex(r'$\mu_{{YZ}} = {:.4f} $'.format(mu_yz_control(comp_states, q_op))))
display(Latex(r'$\mu_{{IY}} = {:.4f} $'.format(mu_yi_control(comp_states, q_op))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [7]:
freq_drive = transm_t.max_freq
eps_drive = 0.5


In [8]:
flx.hamiltonian(as_qobj=True)
flx.charge_op(as_qobj=True)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.-7.40951354e-34j 0.-1.54990962e-01j 0.+5.62829531e-17j
  0.+3.73304538e-01j]
 [0.+1.54990962e-01j 0.+7.66620575e-33j 0.-5.79004448e-01j
  0.-4.52168904e-16j]
 [0.-5.62829531e-17j 0.+5.79004448e-01j 0.+3.36279167e-32j
  0.+6.36100368e-01j]
 [0.-3.73304538e-01j 0.+4.52168904e-16j 0.-6.36100368e-01j
  0.-4.95244653e-33j]]

In [9]:
flx.charge_op(as_qobj=True)[0, 1]

-0.15499096211759397j

In [10]:
# Transmon Hamiltonian
lev_t = 3
lev_f = 4
id_t = qtp.qeye(lev_t)
id_f = qtp.qeye(lev_f)
proj_1_t = qtp.basis(lev_t, 1)*qtp.basis(lev_t, 1).dag()
proj_2_t = qtp.basis(lev_t, 2)*qtp.basis(lev_t, 2).dag()
h_t_single = (transm_t.max_freq - freq_drive)*proj_1_t + (transm_t.anharm + 2*(transm_t.max_freq - freq_drive))*proj_2_t
h_t = qtp.tensor(h_t_single, id_f)

# Fluxonium Hamiltonian
proj_1_f = qtp.basis(lev_f, 1)*qtp.basis(lev_f, 1).dag()
proj_2_f = qtp.basis(lev_f, 2)*qtp.basis(lev_f, 2).dag()
proj_3_f = qtp.basis(lev_f, 3)*qtp.basis(lev_f, 3).dag()
ground_en_f = flx.eig_energies()[0]
h_f_single = flx.hamiltonian(as_qobj=True)[0:lev_f, 0:lev_f] - ground_en_f*id_f - freq_drive*proj_1_f - \
    freq_drive*proj_3_f - 2*freq_drive*proj_2_f
h_f = qtp.tensor(id_t, h_f_single)

# Coupling Hamiltonian
coup_op_t = 1j*transm_t.charge_zpf*(qtp.basis(lev_t, 1)*qtp.basis(lev_t, 0).dag() + np.sqrt(2)*qtp.basis(lev_t, 2)*qtp.basis(lev_t, 1).dag())
q_10_f = flx.charge_op(as_qobj=True)[1, 0]
q_30_f = flx.charge_op(as_qobj=True)[3, 0]
q_21_f = flx.charge_op(as_qobj=True)[2, 1]
coup_op_f = q_10_f*qtp.basis(lev_f, 1)*qtp.basis(lev_f, 0).dag() + \
    q_30_f*qtp.basis(lev_f, 3)*qtp.basis(lev_f, 0).dag() + q_21_f*qtp.basis(lev_f, 2)*qtp.basis(lev_f, 1).dag()
h_coup = jc*(qtp.tensor(coup_op_t, coup_op_f.dag()) + qtp.tensor(coup_op_t, coup_op_f.dag()).dag())
 
# Drive Hamiltonian
h_drive = eps_drive/2*(qtp.tensor(id_t, coup_op_f).dag() + qtp.tensor(id_t, coup_op_f))

#Total Hamiltonian
h_0 = h_t + h_f
h = h_0 + h_coup
h_tot = h + h_drive

In [11]:
eig_en, eig_states = h.eigenstates()
eig_en_tot, eig_states_tot = h_tot.eigenstates()

In [12]:
def dressed_state(
    lev: str,
    hamil: qtp.qobj.Qobj(),
    lev_t: int,
    lev_f: int
):
    """
    Description
    ---------------------------------------------------------
    Function that returns the dressed state
    """
    
    #eig_en_0, eig_states_0 = h_0.eigenstates()
    eig_en, eig_states = hamil.eigenstates()
    
    lev_list = [int(index) for index in lev]
    
    if lev_list[0] >= lev_t or lev_list[1] >= lev_f:
        raise ValueError("Index error: indeces out of bound")
    state_0 = qtp.tensor(qtp.basis(lev_t, lev_list[0]), qtp.basis(lev_f, lev_list[1]))
    #en_0 = eig_en_0[state_index]
    eig_states_list = list(eig_states)
    state = eig_states_list[0]
    en = eig_en[0]
    fid = qtp.fidelity(state*state.dag(), state_0*state_0.dag())
    for state_tmp, en_tmp in zip(eig_states_list, eig_en):
        fid_tmp = qtp.fidelity(state_tmp*state_tmp.dag(), state_0*state_0.dag())
        if fid_tmp > fid:
            state = state_tmp
            fid = fid_tmp
            en = en_tmp
    loc = np.argmax(np.abs(np.array(state)) )
    phase = cmath.phase(state[loc])
    state = np.exp(-1j*phase)*state
    return en, state

To get the effective Hamiltonian we employ numerically an exact Schrieffer-Wolff. All we need to do is define the projectors
involved in the SW transformation

In [13]:
def zz_drive(
    hamil,
    lev_t: int,
    lev_f: int
) -> float:
    en_00 = dressed_state('00', hamil, lev_t, lev_f)[0]
    en_01 = dressed_state('01', hamil, lev_t, lev_f)[0]
    en_10 = dressed_state('10', hamil, lev_t, lev_f)[0]
    en_11 = dressed_state('11', hamil, lev_t, lev_f)[0]
    xi_zz = en_00 + en_11 - en_01 - en_10
    return xi_zz

def project_dressed(
    op: qtp.qobj.Qobj(),
    hamil: qtp.qobj.Qobj(),
    lev_t: int,
    lev_f: int
):
    op_proj = np.zeros([4, 4], dtype=complex)
    states = {}
    states["00"] = dressed_state('00', hamil, lev_t, lev_f)[1]
    states["01"] = dressed_state('01', hamil, lev_t, lev_f)[1]
    states["10"] = dressed_state('10', hamil, lev_t, lev_f)[1]
    states["11"] = dressed_state('11', hamil, lev_t, lev_f)[1]
    
    for key_row, row in zip(states.keys(), range(0, 4)):
        for key_col, col in zip(states.keys(), range(0, 4)):
            op_proj[row, col] = op.matrix_element(states[key_row], states[key_col])
    
    return op_proj

def projector(
    hamil: qtp.qobj.Qobj(),
    lev_t: int,
    lev_f: int,
    labels=["00", "01", "10", "11"]
):
    states = {}
    proj = 0
    for label in labels:
        states[label] = dressed_state(label, hamil, lev_t, lev_f)[1]
        proj += states[label]*states[label].dag()
    return proj, states

In [14]:
proj_dressed, states_dressed = projector(h, lev_t, lev_f)
proj_dressed_drive, states_dressed_drive = projector(h_tot, lev_t, lev_f)
ref_dressed = qtp.tensor(id_t, id_f) - 2*proj_dressed
ref_dressed_drive = qtp.tensor(id_t, id_f) - 2*proj_dressed_drive
u_sw = (ref_dressed*ref_dressed_drive).sqrtm()

In [15]:
def h_sw_exact(
    proj_0: qtp.qobj.Qobj(),
    proj: qtp.qobj.Qobj(),
    hamil: qtp.qobj.Qobj(),
    states_0: dict,
    lev_t: int,
    lev_f: int
):
    refl_0 = qtp.tensor(qtp.qeye(lev_t), qtp.qeye(lev_f)) - 2*proj_0
    refl = qtp.tensor(qtp.qeye(lev_t), qtp.qeye(lev_f)) - 2*proj
    u_sw = (refl_0*refl).sqrtm()
    h_sw = u_sw*hamil*u_sw.dag()
    h_eff = np.zeros([4, 4], dtype=complex)
    for key_row, row in zip(states_0.keys(), range(0, 4)):
        for key_col, col in zip(states_0.keys(), range(0, 4)):
            h_eff[row, col] = hamil.matrix_element(states_0[key_row], states_0[key_col])
    return h_eff

In [16]:
h_sw_eff = h_sw_exact(proj_dressed, proj_dressed_drive, h_tot, states_dressed, lev_t, lev_f)

The easiest thing is to obtain the projector on the dressed basis without the drive

In [17]:
def get_pauli_coeff(
    op: np.ndarray,
    label: str
):
    pauli = {}
    pauli["I"] = np.identity(2, dtype=complex)
    pauli["X"] = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
    pauli["Y"] = np.array([[0.0, -1j*1.0], [1j*1.0, 0.0]], dtype=complex)
    pauli["Z"] = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)
    label_list = [x for x in label]
    pauli_op = np.kron(pauli[label_list[0]], pauli[label_list[1]])
    coeff = 1/4*np.trace(op.conj().T.dot(pauli_op))
    return coeff

In [18]:
get_pauli_coeff(h_sw_eff, "YZ")*1e3

(-0.741328069622359+0j)

In [19]:
mu_yz_control(comp_states, coupled_sys['F'].charge_op())*eps_drive*1e3/2

-0.6909001670163284

In [20]:
def mu_yz_flx_sw(
    transm: Qubit,
    flx: Qubit,
    jc: float
):
    q_zpf = transm.charge_zpf
    omega_t = transm.freq
    omega_flx, states_flx = flx.eig_states(4)
    omega_flx = omega_flx - omega_flx[0]
    q_10 = np.imag(get_mat_elem(flx.charge_op(), states_flx[1], states_flx[0]))
    q_21 = np.imag(get_mat_elem(flx.charge_op(), states_flx[2], states_flx[1]))
    q_30 = np.imag(get_mat_elem(flx.charge_op(), states_flx[3], states_flx[0]))
    coeff = q_21**2/(omega_flx[2] - (omega_t + omega_flx[1]))
    coeff += -q_30**2/(omega_flx[3] - omega_t)
    coeff += q_10**2/(omega_t - omega_flx[1]) 
    mu_yz = jc*q_zpf*coeff/2
    return mu_yz

In [21]:
mu_yz_flx_sw(transm_t, flx, jc)*1e3*eps_drive/2

-0.7551229934306682

There seems to be a mysterious factor of 2

# Generating data

In [22]:
def get_cr_effect(
    data_point: List,
    jc: float,
    flx: Qubit,
    f_03: float,
    f_12: float, 
    q_03: float,
    q_12: float
):
    """
    Description
    -----------------------------------------------------------------------
    Returns the exact cross-resonance coefficient in MHz.
    """
    
    freq = data_point[0]
    eps_ratio = data_point[1]
    
    transm_t = pysqkit.qubits.SimpleTransmon(
            label='T_tmp', 
            max_freq=freq, 
            anharm=-0.3,
            dim_hilbert=3,
        )
    
    min_delta = np.min([np.abs(freq - f_03)/q_03, np.abs(freq - f_12)/q_12])
    eps = eps_ratio*min_delta
    
    freq_drive = freq
    lev_t = 3
    lev_f = 4
    id_t = qtp.qeye(lev_t)
    id_f = qtp.qeye(lev_f)
    proj_1_t = qtp.basis(lev_t, 1)*qtp.basis(lev_t, 1).dag()
    proj_2_t = qtp.basis(lev_t, 2)*qtp.basis(lev_t, 2).dag()
    h_t_single = (transm_t.max_freq - freq_drive)*proj_1_t + (transm_t.anharm + 2*(transm_t.max_freq - freq_drive))*proj_2_t
    h_t = qtp.tensor(h_t_single, id_f)

    # Fluxonium Hamiltonian
    proj_1_f = qtp.basis(lev_f, 1)*qtp.basis(lev_f, 1).dag()
    proj_2_f = qtp.basis(lev_f, 2)*qtp.basis(lev_f, 2).dag()
    proj_3_f = qtp.basis(lev_f, 3)*qtp.basis(lev_f, 3).dag()
    ground_en_f = flx.eig_energies()[0]
    h_f_single = flx.hamiltonian(as_qobj=True) - ground_en_f*id_f - freq_drive*proj_1_f - \
        freq_drive*proj_3_f - 2*freq_drive*proj_2_f
    h_f = qtp.tensor(id_t, h_f_single)

    # Coupling Hamiltonian
    coup_op_t = 1j*transm_t.charge_zpf*(qtp.basis(lev_t, 1)*qtp.basis(lev_t, 0).dag() + np.sqrt(2)*qtp.basis(lev_t, 2)*qtp.basis(lev_t, 1).dag())
    q_10_f = flx.charge_op(as_qobj=True)[1, 0]
    q_30_f = flx.charge_op(as_qobj=True)[3, 0]
    q_21_f = flx.charge_op(as_qobj=True)[2, 1]
    coup_op_f = q_10_f*qtp.basis(lev_f, 1)*qtp.basis(lev_f, 0).dag() + \
        q_30_f*qtp.basis(lev_f, 3)*qtp.basis(lev_f, 0).dag() + q_21_f*qtp.basis(lev_f, 2)*qtp.basis(lev_f, 1).dag()
    h_coup = jc*(qtp.tensor(coup_op_t, coup_op_f.dag()) + qtp.tensor(coup_op_t, coup_op_f.dag()).dag())

    # Drive Hamiltonian
    h_drive = eps/2*(qtp.tensor(id_t, coup_op_f).dag() + qtp.tensor(id_t, coup_op_f))

    #Total Hamiltonian
    h_0 = h_t + h_f
    h = h_0 + h_coup
    h_tot = h + h_drive
    
    proj_dressed, states_dressed = projector(h, lev_t, lev_f)
    proj_dressed_drive, states_dressed_drive = projector(h_tot, lev_t, lev_f)
    ref_dressed = qtp.tensor(id_t, id_f) - 2*proj_dressed
    ref_dressed_drive = qtp.tensor(id_t, id_f) - 2*proj_dressed_drive
    u_sw = (ref_dressed*ref_dressed_drive).sqrtm()
    
    h_sw_eff = h_sw_exact(proj_dressed, proj_dressed_drive, h_tot, states_dressed, lev_t, lev_f)
    
    output = {}
    output["freq_fixed"] = transm_t.freq
    output["freq"] = freq
    output["eps_ratio"] = eps_ratio
    output["eps"] = eps
    output["cr"] = np.real(get_pauli_coeff(h_sw_eff, "YZ")*1e3)
    output["zz"] = np.real(get_pauli_coeff(h_sw_eff, "ZZ")*1e3)
    output["jc"] = jc*1e3
    
    return output

f_03 = eig_flx[3] - eig_flx[0]
q_03 = np.abs(coup_op_f[3, 0])
f_12 = eig_flx[2] - eig_flx[1]
q_12 = np.abs(coup_op_f[2, 1])

func = partial(get_cr_effect, jc=jc, flx=flx, f_03=f_03, f_12=f_12, q_03=q_03, q_12=q_12)
    
    

In [23]:
n_points = 30
n_process = 4
delta_freq = 0.8
ref_freq = parameters_set[p_set]["max_freq_t"]
transm_freq_list = list(np.linspace(ref_freq - delta_freq, ref_freq + delta_freq, n_points))
eps_ratio_max = 0.3
eps_ratio_list = list(np.linspace(0.0, eps_ratio_max, n_points))
iter_list = itertools.product(transm_freq_list, eps_ratio_list)

start = time.time()

pool = multiprocessing.Pool(processes=n_process)

result = pool.map(func, iter_list)

pool.close()
pool.join()

end = time.time()

print("Computation time = {} s".format(end - start))
        
        
        

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-2:
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/ciani/anaconda3/lib/p

KeyboardInterrupt: 

  File "/tmp/ipykernel_44625/2187335619.py", line 66, in get_cr_effect
    proj_dressed_drive, states_dressed_drive = projector(h_tot, lev_t, lev_f)
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/tmp/ipykernel_44625/3620059916.py", line 41, in projector
    states[label] = dressed_state(label, hamil, lev_t, lev_f)[1]
  File "/tmp/ipykernel_44625/821131295.py", line 14, in dressed_state
    eig_en, eig_states = hamil.eigenstates()
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ciani/anaconda3/lib/python3.7/site-packages/qutip/qobj.py", line 1731, in eigenstates
    ekets[:] = [Qobj(vec, dims=new_dims, type=new_type) for vec in evecs]
  File "/home/ciani/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/ciani/anaconda3/lib/python3.7/site-pac

In [ ]:
def plot_cr_effect(
    result
):
    color_map = 'seismic'
    n_points = len(result)
    freq = np.zeros(n_points)
    eps = np.zeros(n_points)
    cr = np.zeros(n_points)
    for k in range(0, n_points):
        freq[k] = result[k]["freq"]
        eps[k] = result[k]["eps_ratio"]
        cr[k] = np.abs(result[k]["cr"])
    freq_vec, eps_vec = np.linspace(freq.min(), \
        freq.max(), n_points), np.linspace(eps.min(), eps.max(), n_points)
    freq_vec, eps_vec = np.meshgrid(freq_vec, eps_vec)
    # Interpolate
    rbf_func = scipy.interpolate.Rbf(freq, eps, cr, function='linear')
    cr_rbf = rbf_func(freq_vec, eps_vec)
    
    aspect = (np.max(freq_vec) - np.min(freq_vec))/(np.max(eps_vec) - np.min(eps_vec))
    fig, ax = plt.subplots(figsize=(6,6))
    im = ax.imshow(cr_rbf, vmin=np.min(cr_rbf), vmax=np.max(cr_rbf), 
                   origin='lower', extent=[np.min(freq_vec), np.max(freq_vec), 
                   np.min(eps_vec), np.max(eps_vec)], 
                   cmap=color_map, aspect=aspect)
    cbar = fig.colorbar(im)
    
    plt.show()

In [ ]:
plot_cr_effect(result)

In [52]:
save = True
if save:
    with open('tmp/data_cr_transmon_fluxonium_strong_drive.txt', 'w') as fp:
        json.dump(result, fp)

In [26]:
my_eps_ratio = 0.2
n_points = 100
n_process = 4
delta_freq = 0.8
transm_freq_list = list(np.linspace(transm_t.freq - delta_freq, transm_t.freq + delta_freq, n_points))
my_list = []
for freq in transm_freq_list:
    my_list.append([freq, my_eps_ratio])




start = time.time()

pool = multiprocessing.Pool(processes=n_process)

result = pool.map(func, my_list)

pool.close()
pool.join()

end = time.time()

print("Computation time = {} s".format(end - start))

Computation time = 90.00986385345459 s


In [27]:
save = True
if save:
    with open('tmp/data_cr_transmon_fluxonium_fixed_drive_ratio.txt', 'w') as fp:
        json.dump(result, fp)

In [47]:
1/(2*np.pi*1.5/1e3)*np.pi/2

166.66666666666669

In [23]:
transm_t.freq

5.0

In [53]:
result[0]

{'freq_fixed': 4.2,
 'freq': 4.2,
 'eps_ratio': 0.0,
 'eps': 0.0,
 'cr': 0.0,
 'zz': 0.0428714567576316,
 'jc': 20.0}